In [1]:
from sklearn.datasets import load_digits

mnist = load_digits()

In [2]:
dir(mnist)

['DESCR', 'data', 'feature_names', 'frame', 'images', 'target', 'target_names']

In [3]:
len(mnist.data)

1797

In [4]:
len(mnist.target)

1797

In [5]:
mnist.target_names

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [6]:
mnist.data[0].shape

(64,)

In [128]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [129]:
from sklearn.preprocessing import OneHotEncoder

mnist_data = mnist.data
mnist_target_sub = mnist.target

ohe = OneHotEncoder(sparse_output=False)
mnist_target = ohe.fit_transform(mnist_target_sub.reshape(-1, 1))

x_train_sub = torch.Tensor(mnist_data[:round(len(mnist_data) * 0.6)])
y_train_sub = torch.Tensor(mnist_target[:round(len(mnist_target) * 0.6)])

x_test_sub = torch.Tensor(mnist_data[round(len(mnist_data) * 0.2):])
y_test_sub = torch.Tensor(mnist_target[round(len(mnist_target) * 0.2):])

x_val_sub = torch.Tensor(mnist_data[round(len(mnist_data) * 0.6):round(len(mnist_data) * 0.6) + round(len(mnist_data) * 0.2)])
y_val_sub = torch.Tensor(mnist_target[round(len(mnist_target) * 0.6):round(len(mnist_target) * 0.6) + round(len(mnist_target) * 0.2)])

In [130]:
train_datase = TensorDataset(x_train_sub, y_train_sub)
test_datase = TensorDataset(x_test_sub, y_test_sub)
val_datase = TensorDataset(x_val_sub, y_val_sub)

In [131]:
train_dataloader = DataLoader(train_datase, batch_size=16, shuffle=True)

In [132]:
class mnist_model(nn.Module):

    def __init__(self):
        super(mnist_model, self).__init__()
        self.lin1 = nn.Linear(64, 32)
        self.lin2 = nn.Linear(32, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.lin1(x)
        x = self.relu(x)
        x = self.lin2(x)
        x = self.softmax(x)
        return x

In [133]:
model = mnist_model()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [134]:
epochs = 20

for epoch in range(epochs + 1):
    for idx, batch in enumerate(train_dataloader):
        # print(batch[1])
        x, y_true = batch

        y_pred = model(x)

        loss = F.cross_entropy(y_pred, y_true)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f'Epoch {epoch + 1}/{epochs} Batch {idx + 1}/{len(train_dataloader)} loss {loss.item()}')

Epoch 1/20 Batch 1/68 loss 3.5343668460845947
Epoch 1/20 Batch 2/68 loss 3.3237380981445312
Epoch 1/20 Batch 3/68 loss 2.6990580558776855
Epoch 1/20 Batch 4/68 loss 2.796438455581665
Epoch 1/20 Batch 5/68 loss 3.20442533493042
Epoch 1/20 Batch 6/68 loss 2.9708218574523926
Epoch 1/20 Batch 7/68 loss 2.1921262741088867
Epoch 1/20 Batch 8/68 loss 2.429425001144409
Epoch 1/20 Batch 9/68 loss 2.183558464050293
Epoch 1/20 Batch 10/68 loss 2.2956702709198
Epoch 1/20 Batch 11/68 loss 2.0661556720733643
Epoch 1/20 Batch 12/68 loss 2.4677438735961914
Epoch 1/20 Batch 13/68 loss 2.2454192638397217
Epoch 1/20 Batch 14/68 loss 2.716817617416382
Epoch 1/20 Batch 15/68 loss 2.322272777557373
Epoch 1/20 Batch 16/68 loss 2.037153482437134
Epoch 1/20 Batch 17/68 loss 2.0255231857299805
Epoch 1/20 Batch 18/68 loss 2.262986183166504
Epoch 1/20 Batch 19/68 loss 2.1233181953430176
Epoch 1/20 Batch 20/68 loss 2.0190696716308594
Epoch 1/20 Batch 21/68 loss 2.1339542865753174
Epoch 1/20 Batch 22/68 loss 1.9694

- 학습 / 테스트
- 정확도
- precision, recall, f1 score
- 틀린 답안이 왜 틀렸는지

In [176]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.3, random_state=42, stratify=mnist.target)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(1257, 64) (540, 64) (1257,) (540,)


In [177]:
class customDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = torch.FloatTensor(x)
        self.y = torch.LongTensor(y)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
train_ds = customDataset(x_train, y_train)
test_ds = customDataset(x_test, y_test)

print(train_ds.__len__())
print(test_ds.__len__())

1257
540


In [197]:
train_dataloader = torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_ds, batch_size=32, shuffle=False)

['_DataLoader__initialized', '_DataLoader__multiprocessing_context', '_IterableDataset_len_called', '__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_auto_collation', '_dataset_kind', '_get_iterator', '_index_sampler', '_iterator', 'batch_sampler', 'batch_size', 'check_worker_number_rationality', 'collate_fn', 'dataset', 'drop_last', 'generator', 'multiprocessing_context', 'num_workers', 'persistent_workers', 'pin_memory', 'pin_memory_device', 'prefetch_factor', 'sampler', 'timeout', 'worker_init_fn']


In [179]:
index, batch = next(enumerate(train_dataloader))
x, y = batch
x.shape

torch.Size([32, 64])

In [181]:
class customModel(nn.Module):
    def __init__(self):
        super(customModel, self).__init__()
        self.linear = nn.Linear(64, 32)
        self.linear2 = nn.Linear(32, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(1) # 1번째 열을 바탕으로 계산한다
    
    def forward(self, x):
        x = self.linear(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x
    
model = customModel()
print(model)
    

customModel(
  (linear): Linear(in_features=64, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=10, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
)


In [195]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.005)
criterion = nn.CrossEntropyLoss()
epochs = 10

def train(model, dataloader, optimizer, loss_func, epochs):
    losses = []
    # 배치에서 데이터 꺼내기
    for epoch in range(epochs):
        for data, target in dataloader:
            # 순전파
            y_pred = model(data)

            # 손실계산
            loss = loss_func(y_pred, target)

            optimizer.zero_grad()
            # 기울기 계산
            loss.backward()
            # 역전파
            optimizer.step()

        print(f'Epoch : {epoch}, Loss : {loss.item()}')
        losses.append(loss.item())
    return losses

In [175]:
train(model, train_dataloader, optimizer, criterion, epochs)

Epoch : 0, Loss : 1.7008533477783203
Epoch : 1, Loss : 1.78902268409729
Epoch : 2, Loss : 1.5882059335708618
Epoch : 3, Loss : 1.5582761764526367
Epoch : 4, Loss : 2.1602888107299805
Epoch : 5, Loss : 1.6518681049346924
Epoch : 6, Loss : 1.8163636922836304
Epoch : 7, Loss : 1.7204962968826294
Epoch : 8, Loss : 1.5911978483200073
Epoch : 9, Loss : 1.7631276845932007


[1.7008533477783203,
 1.78902268409729,
 1.5882059335708618,
 1.5582761764526367,
 2.1602888107299805,
 1.6518681049346924,
 1.8163636922836304,
 1.7204962968826294,
 1.5911978483200073,
 1.7631276845932007]

In [ ]:
def evaluate(model, dataloader, loss_func, epochs):
    losses = []
    # 배치에서 데이터 꺼내기
    for epoch in range(epochs):
        for data, target in dataloader:
            # 순전파
            y_pred = model(data)
            # 손실계산
            loss = loss_func(y_pred, target)

        print(epoch, loss.item())
        losses.append(loss.item())
    return losses

In [185]:
from sklearn.metrics import accuracy_score

In [193]:
def train_validate(model, train_dataloader, test_dataloader, optimizer, loss_func, epochs):
    train_losses = []
    # train_accuracy = []
    validation_losses = []
    # 배치에서 데이터 꺼내기
    for epoch in range(epochs):
        for data, target in train_dataloader:
            # 순전파
            y_pred = model(data)

            # 손실계산
            loss = loss_func(y_pred, target)
            optimizer.zero_grad()
            # 기울기 계산
            loss.backward()
            # 역전파
            optimizer.step()

        print(f'Epoch : {epoch}, Train Loss : {loss.item()}')
        train_losses.append(loss.item())
        # train_accuracy.append(acc)

        for data, target in train_dataloader:
            # 순전파
            y_pred = model(data)

            # 손실계산
            loss = loss_func(y_pred, target)


        print(f'Val Loss : {loss.item()}')
        validation_losses.append(loss.item())

    return train_losses, validation_losses

In [194]:
train_validate(model, train_dataloader, test_dataloader, optimizer, criterion, epochs)

Epoch : 0, Train Loss : 2.3710451126098633
Val Loss : 2.344247817993164
Epoch : 1, Train Loss : 2.3867568969726562
Val Loss : 2.309196949005127
Epoch : 2, Train Loss : 2.3693790435791016
Val Loss : 2.304610252380371
Epoch : 3, Train Loss : 2.357323169708252
Val Loss : 2.3275766372680664
Epoch : 4, Train Loss : 2.3765177726745605
Val Loss : 2.3341686725616455
Epoch : 5, Train Loss : 2.3115053176879883
Val Loss : 2.3611385822296143
Epoch : 6, Train Loss : 2.312469244003296
Val Loss : 2.321190357208252
Epoch : 7, Train Loss : 2.324063777923584
Val Loss : 2.305835723876953
Epoch : 8, Train Loss : 2.3478760719299316
Val Loss : 2.3835740089416504
Epoch : 9, Train Loss : 2.339662551879883
Val Loss : 2.3710224628448486


([2.3710451126098633,
  2.3867568969726562,
  2.3693790435791016,
  2.357323169708252,
  2.3765177726745605,
  2.3115053176879883,
  2.312469244003296,
  2.324063777923584,
  2.3478760719299316,
  2.339662551879883],
 [2.344247817993164,
  2.309196949005127,
  2.304610252380371,
  2.3275766372680664,
  2.3341686725616455,
  2.3611385822296143,
  2.321190357208252,
  2.305835723876953,
  2.3835740089416504,
  2.3710224628448486])